# Fetch Coding Exercise - Data Analysis with DuckDB

This notebook analyzes the JSON data using DuckDB to execute SQL queries and answer business questions.

In [ ]:
import json
import pandas as pd
import duckdb
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from tabulate import tabulate

## 1. Load and Transform Data

In [ ]:
def load_json_data(file_path):
    """Load JSON data from file"""
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line:
                    data.append(json.loads(line))
        return data
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return []

print("Loading data files...")
users_data = load_json_data('data/users.json')
brands_data = load_json_data('data/brands.json')
receipts_data = load_json_data('data/receipts.json')

print(f"Users: {len(users_data)} records")
print(f"Brands: {len(brands_data)} records")
print(f"Receipts: {len(receipts_data)} records")

In [ ]:
def parse_date(date_value):
    """Parse date value from various formats"""
    if not date_value:
        return None
    try:
        if isinstance(date_value, dict) and '$date' in date_value:
            timestamp = date_value['$date']
            if isinstance(timestamp, str):
                return datetime.fromisoformat(timestamp.replace('Z', '+00:00'))
            else:
                return datetime.fromtimestamp(timestamp / 1000)
        elif isinstance(date_value, (int, float)):
            return datetime.fromtimestamp(date_value / 1000)
        elif isinstance(date_value, str):
            return datetime.fromisoformat(date_value.replace('Z', '+00:00'))
        else:
            return None
    except:
        return None

In [ ]:
# Transform users data
print("Processing users data...")
users_df = pd.DataFrame(users_data)
users_df['user_id'] = users_df['_id'].apply(lambda x: x.get('$oid', '') if isinstance(x, dict) else str(x))
users_df['created_date'] = users_df['createdDate'].apply(parse_date)
users_df['last_login'] = users_df['lastLogin'].apply(parse_date)

users = users_df[['user_id', 'state', 'created_date', 'last_login', 'role', 'active', 'signUpSource']].copy() # to avoid any errors, SettingWithCopyWarning is ignored
users.columns = ['user_id', 'state', 'created_date', 'last_login', 'role', 'active', 'sign_up_source']


In [ ]:
# Transform brands data
print("Processing brands data...")
brands_df = pd.DataFrame(brands_data)
brands_df['brand_id'] = brands_df['_id'].apply(lambda x: x.get('$oid', '') if isinstance(x, dict) else str(x))
brands_df['cpg_id'] = brands_df['cpg'].apply(lambda x: x.get('$id', {}).get('$oid', '') if isinstance(x, dict) and '$id' in x else '')

brands = brands_df[['brand_id', 'barcode', 'brandCode', 'category', 'categoryCode', 'name', 'topBrand', 'cpg_id']].copy()
brands.columns = ['brand_id', 'barcode', 'brand_code', 'category', 'category_code', 'name', 'top_brand', 'cpg_id']


In [ ]:
# Transform receipts data
print("Processing receipts data...")
receipts_df = pd.DataFrame(receipts_data)
receipts_df['receipt_id'] = receipts_df['_id'].apply(lambda x: x.get('$oid', '') if isinstance(x, dict) else str(x))
receipts_df['date_scanned'] = receipts_df['dateScanned'].apply(parse_date)
receipts_df['purchase_date'] = receipts_df['purchaseDate'].apply(parse_date)
receipts_df['create_date'] = receipts_df['createDate'].apply(parse_date)
receipts_df['finished_date'] = receipts_df['finishedDate'].apply(parse_date)
receipts_df['modify_date'] = receipts_df['modifyDate'].apply(parse_date)
receipts_df['points_awarded_date'] = receipts_df['pointsAwardedDate'].apply(parse_date)

receipts = receipts_df[['receipt_id', 'userId', 'date_scanned', 'purchase_date', 'totalSpent', 
                                'pointsEarned', 'bonusPointsEarned', 'bonusPointsEarnedReason', 
                                'rewardsReceiptStatus', 'purchasedItemCount', 'create_date', 
                                'finished_date', 'modify_date', 'points_awarded_date']].copy()
receipts.columns = ['receipt_id', 'user_id', 'date_scanned', 'purchase_date', 'total_spent', 
                            'points_earned', 'bonus_points', 'bonus_points_reason', 'status', 'item_count',
                            'create_date', 'finished_date', 'modify_date', 'points_awarded_date']


In [ ]:
# Transform receipt items data
print("Processing receipt items...")
receipt_items = []
for receipt in receipts_data:
    receipt_id = receipt['_id']['$oid'] if isinstance(receipt['_id'], dict) else str(receipt['_id'])
    if 'rewardsReceiptItemList' in receipt:
        for i, item in enumerate(receipt['rewardsReceiptItemList']):
            def safe_float(val):
                try:
                    if val == '' or val is None:
                        return None
                    return float(val)
                except:
                    return None
            item_data = {
                'item_id': f"{receipt_id}_{i}",
                'receipt_id': receipt_id,
                'brand_code': item.get('brandCode', ''),
                'barcode': item.get('barcode', ''),
                'description': item.get('description', ''),
                'final_price': safe_float(item.get('finalPrice', '')),
                'item_price': safe_float(item.get('itemPrice', '')),
                'quantity': item.get('quantityPurchased', 0),
                'partner_item_id': item.get('partnerItemId', ''),
                'needs_fetch_review': item.get('needsFetchReview', False),
                'prevent_target_gap_points': item.get('preventTargetGapPoints', False),
                'user_flagged_barcode': item.get('userFlaggedBarcode', ''),
                'user_flagged_new_item': item.get('userFlaggedNewItem', False),
                'user_flagged_price': safe_float(item.get('userFlaggedPrice', '')),
                'user_flagged_quantity': item.get('userFlaggedQuantity', 0)
            }
            receipt_items.append(item_data)
    else:
        item_data = {
            'item_id': f"{receipt_id}_{i}",
            'receipt_id': receipt_id,
            'brand_id': None,
            'barcode': None,
            'description': '',
            'final_price': None,
            'item_price': None,
            'quantity': 0,
            'partner_item_id': '',
            'needs_fetch_review': False,
            'prevent_target_gap_points': False,
            'user_flagged_barcode': '',
            'user_flagged_new_item': False,
            'user_flagged_price': None,
            'user_flagged_quantity': 0
        }
        receipt_items.append(item_data)

receipt_items = pd.DataFrame(receipt_items)
print(f"Receipt items: {len(receipt_items_df)} items")
receipt_items.head()

## 2. Create DuckDB Database and Load Data

In [ ]:
# Create DuckDB connection
con = duckdb.connect(':memory:')

# Register DataFrames as tables
con.register('users', users)
con.register('brands', brands)
con.register('receipts', receipts)
con.register('receipt_items', receipt_items)

print("Tables created in DuckDB:")
print(con.execute("SHOW TABLES").fetchall())

## 3. Data Quality Analysis

In [ ]:
# Check for duplicate users
print("=== DUPLICATE USERS ===")
duplicate_users = con.execute("""
    SELECT user_id, COUNT(*) as duplicate_count
    FROM users
    GROUP BY user_id
    HAVING COUNT(*) > 1
""").fetchall()
print(f"Duplicate user records: {len(duplicate_users)}")

# Sample duplicate users
if duplicate_users:
    print("Sample duplicate users:")
    for user in duplicate_users[:5]:
        print(user) # Display first 5 duplicate users

 # Analysis duplicate users
print("\n=== DUPLICATE USERS ANALYSIS ===")
duplicate_user_ids = [user[0] for user in duplicate_users]
if duplicate_user_ids:
    duplicate_user_data = con.execute(f"""
        SELECT *
        FROM users
        WHERE user_id IN ({', '.join(['?'] * len(duplicate_user_ids))})
    """, duplicate_user_ids).fetchall()[:15]  # Limit to first few records for display

    print(tabulate( duplicate_user_data)) # Looks like  records are duplicated based on user_id with same values


# Check for missing user data
print("\n=== MISSING USER DATA ===")
missing_last_login = con.execute("SELECT COUNT(*) FROM users WHERE last_login IS NULL").fetchone()[0]
missing_state = con.execute("SELECT COUNT(*) FROM users WHERE state IS NULL OR state = ''").fetchone()[0]
missing_sign_up_source = con.execute("SELECT COUNT(*) FROM users WHERE sign_up_source IS NULL OR sign_up_source = ''").fetchone()[0]
missing_created_date = con.execute("SELECT COUNT(*) FROM users WHERE created_date IS NULL ").fetchone()[0]
print(f"Users missing last_login: {missing_last_login}")
print(f"Users missing state: {missing_state}")
print(f"Users missing sign_up_source: {missing_sign_up_source}")
print(f"Users missing created_date: {missing_created_date}")



In [ ]:
# Check top 5 records from brands
print("\n=== TOP 5 BRANDS ===")
top_brands = con.execute("SELECT * FROM brands LIMIT 5").fetchall()
print(tabulate(top_brands, headers=['brand_id', 'barcode', 'brand_code', 'category', 'category_code', 'name', 'top_brand', 'cpg_id'])) 

# Check for duplicate barnd ids in brands
print("\n=== DUPLICATE BRAND IDS ===")
duplicate_brand_ids = con.execute("""
    SELECT brand_id, COUNT(*) as duplicate_count
    FROM brands
    GROUP BY brand_id
    HAVING COUNT(*) > 1
    LIMIT 10
""").fetchall()
print(f"Duplicate brand IDs found: {len(duplicate_brand_ids)}")

# check for brand id and brand code relationship
print("\n=== BRAND ID AND BRAND CODE RELATIONSHIP ===")
brand_id_code_relationship = con.execute("""
    SELECT brand_id, brand_code, COUNT(*) as count
    FROM brands
    GROUP BY brand_id, brand_code
    HAVING COUNT(*) > 1
    LIMIT 10
""").fetchall() # Display brand_id and brand_code relationship
print(f"Brand ID and Brand Code relationship found 1: {len(brand_id_code_relationship)}")
print(tabulate(brand_id_code_relationship, headers=['brand_id', 'brand_code', 'count'])) 

brand_id_code_relationship = con.execute("""
    SELECT brand_id, count(distinct brand_code) as count
    FROM brands
    GROUP BY brand_id
    HAVING COUNT(*) > 1
    LIMIT 10
""").fetchall()
print(f"Brand ID and Brand Code relationship found: {len(brand_id_code_relationship)}")
print(tabulate(brand_id_code_relationship, headers=['brand_id', 'count'])) 

   

brand_id_code_relationship = con.execute("""
    SELECT brand_id, count(distinct brand_code) as count
    FROM brands
    GROUP BY brand_id
    HAVING count = 0
    LIMIT 10
""").fetchall()
print(f"Brand ID and Brand Code relationship found 1: {len(brand_id_code_relationship)}")
print(tabulate(brand_id_code_relationship, headers=['brand_id', 'count']))

brand_id_code_relationship = con.execute("""
    SELECT brand_code, count(distinct brand_id) as count
    FROM brands
    GROUP BY brand_code
    HAVING COUNT(*) > 1
    LIMIT 10
""").fetchall()
print(f"Brand ID and Brand Code relationship found 2: {len(brand_id_code_relationship)}")
print(tabulate(brand_id_code_relationship, headers=[ 'brand_code', 'count']))

print("\n=== BRAND ID AND BRAND CODE RELATIONSHIP Notes ===\n" \
"1. Brand id is unique \n" \
"2. Brand ID usually have a brand code and if present relationship 1:1 or its missing" \
)

# Check for duplicate barcodes in brands
print("\n=== DUPLICATE BRAND BARCODES ===")
duplicate_barcodes = con.execute("""
    SELECT barcode, COUNT(*) as duplicate_count
    FROM brands
    WHERE barcode IS NOT NULL AND barcode != ''
    GROUP BY barcode
    HAVING COUNT(*) > 1
    LIMIT 10
""").fetchall()
print(f"Duplicate barcodes found: {len(duplicate_barcodes)}")
print(tabulate(duplicate_barcodes, headers=['barcode', 'duplicate_count']))

# Displkay duplicate barcodes
if duplicate_barcodes:
    print("\nSample duplicate barcodes:")
    duplicate_barcodes_sample = con.execute( f"""
        SELECT *
        FROM brands
        WHERE barcode IN ({', '.join(['?'] * len(duplicate_barcodes))})
        Order by barcode
    """, [barcode[0] for barcode in duplicate_barcodes]).fetchall()[:15]  # Display first 15 records
    print(tabulate(duplicate_barcodes_sample, headers=['brand_id', 'barcode', 'brand_code', 'category', 'category_code', 'name', 'top_brand', 'cpg_id']))


# Check for test data in brands
print("\n=== TEST DATA IN BRANDS ===")
test_brands = con.execute("SELECT case when name like '%test%' then 'test' else name end as name, " \
"COUNT(*) count FROM brands group by case when name like '%test%' then 'test' else name end " \
"having count >1 order by count desc").fetchall()
print(tabulate(test_brands, headers=['name', 'count'])) # Display test brands`)

# Investigate duplicate Names
print("\n=== DUPLICATE BRAND NAMES ===")
duplicate_names = con.execute("""
    SELECT * from brands where name in (
    SELECT name 
    FROM brands
    where name  not like '%test%'
     GROUP BY name
    HAVING COUNT(*) > 1 ) order by name
""").fetchall()
print(tabulate(duplicate_names, headers=['brand_id', 'barcode', 'brand_code', 'category', 'category_code', 'name', 'top_brand', 'cpg_id'])) # Display duplicate names



In [ ]:
# Check top 10 records from receipts

# receipts columns: receipt_id, user_id, date_scanned, purchase_date, total_spent, points_earned, bonus_points, bonus_points_reason, status, item_count
con.execute("SELECT * FROM receipts LIMIT 1").fetchall()
receipt_columns = [desc[0] for desc in con.description]

# Total receipts
total_receipts = con.execute("SELECT COUNT(*) FROM receipts").fetchone()[0]
print(f"\nTotal receipts: {total_receipts}")

print("\n=== TOP 10 RECEIPTS ===")
top_receipts = con.execute("SELECT * FROM receipts LIMIT 10").fetchall()
print(tabulate(top_receipts, headers=receipt_columns)) # Display top 10 receipts

# Check for duplicate in receipt ids
print("\n=== DUPLICATE RECEIPT IDS ===")
duplicate_receipt_ids = con.execute("""
    SELECT receipt_id, COUNT(*) as duplicate_count
    FROM receipts
    GROUP BY receipt_id
    HAVING COUNT(*) > 1
    LIMIT 10
""").fetchall()
print(f"Duplicate receipt IDs found: {len(duplicate_receipt_ids)}")
if duplicate_receipt_ids:
    print("Sample duplicate receipt IDs:")
    for receipt in duplicate_receipt_ids[:5]:
        print(receipt) # Display first 5 duplicate receipt IDs 

# Check for missing receipt data
print("\n=== MISSING RECEIPT DATA ===")

for column in receipt_columns:
    missing_count = con.execute(f"SELECT COUNT(*) FROM receipts WHERE {column} IS NULL").fetchone()[0]
    print(f"Missing {column} count: {missing_count}")


In [ ]:
# Check receipt items table

print("\n=== RECEIPT ITEMS TABLE ===")
receipt_items = con.execute("SELECT * FROM receipt_items LIMIT 10").fetchall()
receipt_items_columns = [desc[0] for desc in con.description]
print(tabulate(receipt_items, headers=receipt_items_columns)) # Display first 11 receipt items

# Check cnt receipt id in receipt items is same as receipts
print("\n=== RECEIPT ID COUNT IN RECEIPT ITEMS ===")
receipt_id_count_items = con.execute("""
    SELECT count(distinct receipt_id) as receipts_count
    FROM receipt_items
    
""").fetchall()
receipt_id_count_receipts = con.execute("""
    SELECT count(distinct receipt_id) as receipts_count
    FROM receipts
    
""").fetchall()
print(f"Do we have same count? : {receipt_id_count_items[0][0] == receipt_id_count_receipts[0][0]}")

# Check item count in receipt is same as count of items per receipt id in receipt items
print("\n=== ITEM COUNT IN RECEIPTS ===")
item_count_in_receipts = con.execute("""
    SELECT r.receipt_id, item_count, COUNT(*) as items_count
    FROM receipts r
    JOIN receipt_items ri ON r.receipt_id = ri.receipt_id
    GROUP BY r.receipt_id, item_count
    HAVING COUNT(*) != item_count
    LIMIT 10
""").fetchall()
if item_count_in_receipts:
    print(f"Item count mismatch found: {len(item_count_in_receipts)}")
    print(tabulate(item_count_in_receipts, headers=['receipt_id', 'item_count', 'items_count']))
else:
    print("No item count mismatch found in receipts.")  

# Drilling down on receipt id 60088a100a7214ad890000bd

print("\n=== DRILLING DOWN ON RECEIPT ID 60088a100a7214ad890000bd ===")
receipt_id = '60088a100a7214ad890000bd'
receipt_details = con.execute(f""" select * from receipts where receipt_id = '{receipt_id}'""").fetchall()
print("Receipt Details:")
print(tabulate(receipt_details, headers=receipt_columns)) # Display receipt details
print("\nReceipt Items Details:")
receipt_items_details = con.execute(f"""
    SELECT *
    FROM receipt_items
    WHERE receipt_id = '{receipt_id}'
""").fetchall()

# print(tabulate(receipt_items_details, headers=receipt_items_columns)) # Display receipt items details

# Check for missing receipt items values
print("\n=== MISSING RECEIPT ITEMS VALUES ===")
for column in receipt_items_columns:
    missing_count = con.execute(f"SELECT COUNT(*) FROM receipt_items WHERE {column} IS NULL").fetchone()[0]
    print(f"Missing {column} count: {missing_count}")   

## Summary of Data Quality Issues found
### Users
- Duplicate user records: **70**
- Users missing `last_login`: **62**
- Users missing `state`: **56**
- Users missing `sign_up_source`: **48**

**Next Steps:** Remove duplicates.

---

### Brands
Brands have multiple issues:
- `brand_id` and `brand_code` are distinct; if `brand_code` is present, it has a one-to-one mapping with `brand_id`. Otherwise, it's missing.
- Duplicate barcodes exist — the same barcode appears for different `brand_id` and `name` combinations.
- There are **429** test brand names present.

**Next Steps:** Assuming barcodes should be unique, we will deduplicate and retain the first occurrence as the reference for each `brand_id`.

---

### Receipts
- Missing `purchase_date`: **448**
- Missing `total_spent`: **435**
- Missing `bonus_points`: **575**
- Missing `points_earned`: **510**
- Missing `bonus_points_reason`: **575**
- Missing `finished_date`: **551**
- Missing `item_count`: **484**
- Missing `points_awarded_date`: **582**

---

### Receipt Items
- Missing `brand_code`: **440**
- Missing `final_price`: **614**
- Missing `item_price`: **614**
- Missing `user_flagged_price`: **7,082**


## 3.5. Cleaning the Data

In [ ]:
# Removing duplicate users
print("\n=== REMOVING DUPLICATE USERS ===")
con.execute("""
    CREATE OR REPLACE Table users_clean AS
    SELECT DISTINCT ON (user_id) *
    FROM users
    ORDER BY user_id, created_date DESC
""")

# Test if users_clean has no duplicate user_ids
duplicate_users_clean = con.execute("""
    SELECT user_id, COUNT(*) as duplicate_count
    FROM users_clean
    GROUP BY user_id
    HAVING COUNT(*) > 1
    LIMIT 10
""").fetchall()
if duplicate_users_clean:
    print(f"Duplicate user IDs found in users_clean: {len(duplicate_users_clean)}")
else:
    print("No duplicate user IDs found in users_clean.")






# Remove duplicate barcodes in brands
print("\n=== REMOVING DUPLICATE BARCODES IN BRANDS ===")
con.execute("""
    CREATE OR REPLACE TABLE brands_clean AS
    SELECT brands.*, row_number() OVER (PARTITION BY barcode ORDER BY brand_id) as rn
    FROM brands
    ORDER BY barcode, brand_id
""")

# Test if brands_clean has no duplicate barcodes
duplicate_barcodes_clean = con.execute("""
    SELECT barcode, COUNT(*) as duplicate_count
    FROM brands_clean
    WHERE barcode IS NOT NULL AND barcode != ''
    
    and rn = 1
    GROUP BY barcode
    HAVING COUNT(*) > 1
    LIMIT 10""").fetchall()
if duplicate_barcodes_clean:
    print(f"Duplicate barcodes found in brands_clean: {len(duplicate_barcodes_clean)}")
else:
    print("No duplicate barcodes found in brands_clean.")

# Drop duplicate barcodes in brands
print("\n=== DROPPING DUPLICATE BARCODES IN BRANDS ===")
con.execute("DELETE FROM brands_clean WHERE rn > 1")
con.execute("ALTER TABLE brands_clean DROP COLUMN rn")
# Rename brands_clean to brands
con.execute("ALTER TABLE brands_clean RENAME TO brands")    



In [ ]:





print("=== TOP 5 BRANDS BY RECEIPTS SCANNED (RECENT MONTH) ===")
top_brands_query = """
with monthly_brand_stats as (
SELECT 
    b.brand_id,
    b.name as brand_name,
    COUNT(DISTINCT r.receipt_id) as receipts_scanned,
    ROW_NUMBER() OVER (ORDER BY COUNT(DISTINCT r.receipt_id) DESC) as rank
    FROM brands b
    JOIN receipt_items ri ON b.brand_code = ri.brand_code
    JOIN receipts r ON ri.receipt_id = r.receipt_id
    WHERE DATE_TRUNC('month', r.date_scanned) = (
        SELECT DATE_TRUNC('month', MAX(date_scanned) ) FROM receipts
    )
    GROUP BY b.brand_id, b.name
)
SELECT 
    brand_name,
    receipts_scanned,
    rank
FROM monthly_brand_stats
WHERE rank <= 5
ORDER BY rank;
"""

try:
    top_brands_result = con.execute(top_brands_query).fetchall()
    if top_brands_result:
        for brand_name, receipts_scanned, rank in top_brands_result:
            print(f"{rank}. {brand_name}: {receipts_scanned} receipts")
    else:
        print("No data found for the specified time period")
except Exception as e:
    print(f"Error executing query: {e}")

In [ ]:
# Query 1: # Top 5 Brands by Receipts Scanned for Most Recent Month

# Get most recent month
print("\n=== MOST RECENT MONTH) ===")


print("Most recent month:", con.execute("""
    select date_trunc('month', max(date_scanned)) as most_recent_month from receipts
""").fetchone()[0])

top_brands_query = """
WITH latest_receipts AS (
    SELECT DISTINCT 
        r.receipt_id, 
        r.date_scanned, 
        ri.brand_code, 
        b.name AS brand_name
    FROM receipts r
    JOIN receipt_items ri 
        ON r.receipt_id = ri.receipt_id
    LEFT JOIN brands b 
        ON ri.brand_code = b.brand_code
        AND ri.brand_code IS NOT NULL
        and ri.brand_code <> ''
    WHERE DATE_TRUNC('month', r.date_scanned) = (
        SELECT DATE_TRUNC('month', MAX(date_scanned)) FROM receipts
    )
)
--- Top 5 Brands by Receipts Scanned in Latest Month
SELECT 
    brand_name, 
    COUNT(DISTINCT receipt_id) AS cnt
FROM latest_receipts
GROUP BY brand_name
ORDER BY cnt DESC
LIMIT 5;
;"""
# Check Latest month receipts
print("\n=== LATEST MONTH RECEIPTS ===")
top_brands = con.execute(top_brands_query).fetchall()

print(tabulate(top_brands, headers=['Brand Name', 'Receipts Count']))  
print("""No brand information could be found for the latest month's receipts, as brand_code is missing from those records. """ )

In [ ]:
# Query 2: Compare Top 5 Brands by Receipts Scanned (Recent Month vs Previous Month)
print("=== TOP 5 BRANDS BY RECEIPTS SCANNED: RECENT MONTH VS PREVIOUS MONTH ===")

compare_brands_query = """
WITH months AS (
    SELECT 
        DATE_TRUNC('month', MAX(date_scanned)) AS recent_month,
        DATE_TRUNC('month', MAX(date_scanned - - INTERVAL 1 MONTH))  AS prev_month
    FROM receipts
),
brand_stats AS (
    SELECT 
        b.brand_id, ri.brand_code,
        b.name AS brand_name,
        DATE_TRUNC('month', r.date_scanned) AS month,
        COUNT(DISTINCT r.receipt_id) AS receipts_scanned
    FROM brands b
    JOIN receipt_items ri ON b.brand_code = ri.brand_code
    JOIN receipts r ON ri.receipt_id = r.receipt_id
    and (ri.brand_code IS NOT NULL and ri.brand_code != '')
    WHERE DATE_TRUNC('month', r.date_scanned) IN (SELECT recent_month FROM months UNION SELECT prev_month FROM months)
    GROUP BY b.brand_id, ri.brand_code, b.name, month
),
ranked_brands AS (
    SELECT 
        brand_name,
        month,
        receipts_scanned,
        ROW_NUMBER() OVER (PARTITION BY month ORDER BY receipts_scanned DESC) AS rank
    FROM brand_stats
)
SELECT 
    rb_recent.brand_name,
    rb_recent.receipts_scanned AS recent_month_scanned,
    rb_recent.rank AS recent_month_rank,
    rb_prev.receipts_scanned AS prev_month_scanned,
    rb_prev.rank AS prev_month_rank
FROM 
    (SELECT * FROM ranked_brands WHERE month = (SELECT recent_month FROM months) AND rank <= 5) rb_recent
LEFT JOIN 
    (SELECT * FROM ranked_brands WHERE month = (SELECT prev_month FROM months)) rb_prev
ON rb_recent.brand_name = rb_prev.brand_name
ORDER BY rb_recent.rank;

"""
top_brand_comparison= con.execute(compare_brands_query).fetchall()
print(tabulate(top_brand_comparison, headers='keys', tablefmt='psql'))
print("""No brand information could be found for the latest and previous months because brand_code is missing """ )

In [ ]:
# Diving deeper into brand_code distribution

brand_code_dist_query = """ select 
    case when brand_code is null or brand_code = '' then 'Unknown' else brand_code end as brand_code,
    date_trunc('month', date_scanned) as month, 
    count(DISTINCT receipt_id) as receipts_scanned
from receipt_items join receipts using (receipt_id)
--where receipt_id in (select receipt_id from receipts where date_scanned >= '2021-02-01'::date)
group by 1,2
order by  2 desc, 3 desc, 1; """

brand_code_dist = con.execute(brand_code_dist_query).fetchall()
print(tabulate(brand_code_dist, headers=['Brand Code', 'Month', 'Count'], tablefmt='psql'))

In [ ]:
# Checking the status of receipts in the database
query = """
SELECT status, COUNT(*) AS count
FROM receipts
GROUP BY status
ORDER BY count DESC;
"""
status_counts = con.execute(query).fetchall()
print(tabulate(status_counts, headers=['Status', 'Count'], tablefmt='psql'))

In [ ]:
# Query 3: Average Spend Comparison (Accepted vs Rejected Receipts)
print("=== AVERAGE SPEND COMPARISON (ACCEPTED VS REJECTED) ===")
avg_spend_query = """
SELECT 
  status,
  COUNT(*) as receipt_count,
  AVG(CAST(total_spent AS DOUBLE)) as avg_spend,
  SUM(CAST(total_spent AS DOUBLE)) as total_spend
FROM receipts
WHERE status IN ('FINISHED', 'REJECTED')
GROUP BY status
ORDER BY avg_spend DESC;
"""

try:
    avg_spend_result = con.execute(avg_spend_query).fetchall()
    for status, receipt_count, avg_spend, total_spend in avg_spend_result:
        print(f"{status}: {receipt_count} receipts, avg spend: ${avg_spend:.2f}, total: ${total_spend:.2f}")
except Exception as e:
    print(f"Error executing query: {e}")

In [ ]:
# Query 4: Total Items Purchased Comparison (Accepted vs Rejected)
print("=== TOTAL ITEMS PURCHASED COMPARISON (ACCEPTED VS REJECTED) ===")
items_query = """
SELECT 
  r.status,
  COUNT(*) as receipt_count,
  SUM(r.item_count) as total_items_purchased,
  AVG(r.item_count) as avg_items_per_receipt
FROM receipts r
WHERE r.status IN ('FINISHED', 'REJECTED')
GROUP BY r.status
ORDER BY total_items_purchased DESC;
"""

try:
    items_result = con.execute(items_query).fetchall()
    for status, receipt_count, total_items, avg_items in items_result:
        print(f"{status}: {receipt_count} receipts, {total_items} total items, avg {avg_items:.1f} items per receipt")
except Exception as e:
    print(f"Error executing query: {e}")

In [ ]:
# Check Max creation date in users
print("\n=== MAX CREATION DATE IN USERS ===")
max_creation_date = con.execute("""
    SELECT MAX(created_date) FROM users
""").fetchone()[0]
if max_creation_date:
    print(f"Max creation date in users: {max_creation_date}")
else:
    print("No creation date found in users table.")

In [ ]:
# Query 5: Brand with Most Spend Among Recent Users (6 months)
print("=== BRAND WITH MOST SPEND AMONG RECENT USERS (6 MONTHS) ===")
recent_users_query = """
SELECT 
  b.name as brand_name,
  SUM(CAST(ri.final_price AS DOUBLE)) as total_spend
FROM brands b
JOIN receipt_items ri ON b.brand_code = ri.brand_code
JOIN receipts r ON ri.receipt_id = r.receipt_id
JOIN users u ON r.user_id = u.user_id
WHERE u.created_date >= CURRENT_DATE - INTERVAL 6 MONTHS
--- WHERE u.created_date >= '2021-02-12'::date - INTERVAL 6 MONTHS --- for testing

GROUP BY b.name
ORDER BY total_spend DESC
LIMIT 1;
"""

try:
    recent_users_result = con.execute(recent_users_query).fetchall()
    if recent_users_result:
        brand_name, total_spend = recent_users_result[0]
        print(f"Top brand: {brand_name}")
        print(f"Total spend: ${total_spend:.2f}")
    else:
        print("No data found for recent users")
except Exception as e:
    print(f"Error executing query: {e}")

In [ ]:
# Query 6: Brand has the most transactions among users who were created within the past 6 months?
print("=== BRAND WITH MOST TRANSACTIONS AMONG RECENT USERS (6 MONTHS) ===")
recent_users_query = """
SELECT 
  b.name as brand_name,
  COUNT(DISTINCT r.receipt_id) as receipt_count
FROM brands b
JOIN receipt_items ri ON b.brand_code = ri.brand_code
JOIN receipts r ON ri.receipt_id = r.receipt_id
JOIN users u ON r.user_id = u.user_id
-- WHERE u.created_date >= CURRENT_DATE - INTERVAL 6 MONTHS
WHERE u.created_date >= '2021-02-12'::date - INTERVAL 6 MONTHS -- for testing

GROUP BY b.name
ORDER BY receipt_count DESC
LIMIT 1;"""

try:
    recent_users_result = con.execute(recent_users_query).fetchall()
    if recent_users_result:
        brand_name,  receipt_count = recent_users_result[0]
        print(f"Top brand: {brand_name}")
        print(f"Receipt count: {receipt_count}")
    else:
        print("No data found for recent users")
except Exception as e:
    print(f"Error executing query: {e}")

In [ ]:
# Close the connection
con.close()
print("\nAnalysis complete! Database connection closed.")